In [9]:
import matplotlib

from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [10]:
#resizing images
IMAGE_SIZE = [224, 224]

train_path = 'chest_xray/train'
valid_path ='chest_xray/test'

#STEPS
#path to images: train and test
# instatiate the network model
# remove the start and top layers
# skip training the weights
#define no.of classes =>folders= glob(dataset/train)
#Flatten model output
#Add last (prediction) softmax layer
# instatiate model and see its structure
# compile the model
# preprocessing images with ImageDataGenerator-create datagen,then create a flow directory
# fit the model
# train it
# measure it performance

In [11]:
#importing the vgg16 library. then add preprocessing layer.
#also usinf imagenet weights
vgg = VGG16(input_shape= IMAGE_SIZE + [3], weights= 'imagenet', include_top = False)
#The original last layer of VGG and resnet is always 1000 categories. Here we are skipping becoz
#our dataset has only 2 categories; normal and pneumonia
#we are also skipping the first layer becoz we are defining our our IMAGE_SIZE of 224*224
#so include_top = False removes first and last layers

In [12]:
#skip training of layers to maintain our imported imagenet weights
for layer in vgg.layers:
    layer.trainable = False

In [20]:
#Defining number of output classes
folders = glob('chest_xray/train/*')
folders

['chest_xray/train\\NORMAL', 'chest_xray/train\\PNEUMONIA']

In [21]:
#flattening output
x= Flatten()(vgg.output)

In [22]:
#add last output softmax layer
prediction = Dense(len(folders), activation = 'softmax')(x) 
#folders is the number of output classes of the network

In [23]:
#model object
model = Model(inputs = vgg.input, outputs = prediction)

#structure of the model
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [24]:
#compiling the model
model.compile(loss ='categorical_crossentropy',
             optimizer = 'adam',
              metrics = ['accuracy']
             )

In [26]:
#using DataGenerator to create variance of test data
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                   zoom_range =0.2,
                                   horizontal_flip = True
                                  )

test_datagen = ImageDataGenerator(rescale = 1./255)

In [33]:
#Inserting images for train and test
training_set = train_datagen.flow_from_directory('chest_xray/train',
                                               target_size = (224, 224),
                                               batch_size = 32,
                                               class_mode = 'categorical'
                                              )

testing_set = test_datagen.flow_from_directory('chest_xray/test',
                                               target_size = (224, 224),
                                               batch_size = 32,
                                               class_mode = 'categorical'
                                              )


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [34]:
#Fit the model

r= model.fit_generator(
    training_set,
    validation_data = testing_set,
    epochs = 2,
    steps_per_epoch = len(training_set),
    validation_steps = len(testing_set)
)

Epoch 1/2
163/163 [==============================] - 979s 6s/step - loss: 0.2114 - accuracy: 0.9222 - val_loss: 0.4533 - val_accuracy: 0.8846
Epoch 2/2
163/163 [==============================] - 896s 5s/step - loss: 0.1125 - accuracy: 0.9569 - val_loss: 0.2590 - val_accuracy: 0.9103


In [36]:
#plotting loss
plt.plot(r.history['loss'], label = 'train loss')
plt.plot(r.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')


C:\Users\Kweronda\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  """


In [41]:
#saving model
import tensorflow as tf
from keras.models import load_model
model.save('xray_vgg16.m1')

In [45]:
#Validating model accuracy

from keras.models import load_model

model = load_model('xray_vgg16.m1')
img = image.load_img('chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg', target_size = (224, 224))
x= image.img_to_array(img)
x= np.expand_dims(x, axis = 0)
img_data = preprocess_input(x)
classes = model.predict(img_data)
print(classes)

[[1. 0.]]


In [46]:
#model predicted the image is for a normal patient, and it indeed is.